In [3]:
import os
import re
from dotenv import load_dotenv
from typing import Literal

from langchain_core.documents import Document
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import MessagesState

# .env 파일에서 환경 변수 로드
load_dotenv()

True

#### 문제 4-2 : 조건부 분기가 있는 메뉴 추천 시스템 ( LangGraph 사용)

In [4]:
# 1. 벡터 DB 로드
# 이전 문제에서 생성한 FAISS 벡터 DB를 로드합니다.
if not os.path.exists("../db/cafe_db"):
    raise FileNotFoundError("Cafe DB not found. Please run the previous notebook (4-1) to create it.")

embeddings_model = OpenAIEmbeddings()
cafe_db = FAISS.load_local(
    "../db/cafe_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

print("카페 메뉴 벡터 DB 로딩 완료!")

# 2. 고급 정보 추출 함수 (요구사항)
def extract_menu_info(doc: Document) -> dict:
    """Vector DB 문서에서 구조화된 메뉴 정보 추출"""
    content = doc.page_content
    menu_name = doc.metadata.get('menu_name', 'Unknown')
    
    # 정규표현식으로 가격, 설명 등 추출
    price_match = re.search(r'가격:\s*(₩[\d,]+)', content)
    description_match = re.search(r'설명:\s*(.+)', content, re.DOTALL)
    
    return {
        "name": menu_name,
        "price": price_match.group(1) if price_match else "가격 정보 없음",
        "description": description_match.group(1).strip() if description_match else "설명 없음"
    }

# 3. 문의 유형 분류 함수
def classify_question(user_message: str) -> Literal["price", "recommendation", "menu", "general"]:
    """키워드 기반으로 사용자 문의 유형을 분류합니다."""
    if any(keyword in user_message for keyword in ["가격", "얼마"]):
        return "price"
    elif any(keyword in user_message for keyword in ["추천", "뭐가 맛있어", "인기"]):
        return "recommendation"
    elif len(user_message.split()) < 3 and ("안녕" in user_message or "hi" in user_message):
        return "general"
    else:
        return "menu"


카페 메뉴 벡터 DB 로딩 완료!


In [5]:
def handle_menu_inquiry(state: MessagesState) -> dict:
    """특정 메뉴에 대한 문의를 처리하고 상세 정보를 반환합니다."""
    user_message = state["messages"][-1].content
    # 사용자 메시지를 직접 검색어로 사용하여 의미론적 검색 수행
    docs = cafe_db.similarity_search(user_message, k=2)
    
    if not docs:
        response_text = "죄송합니다, 요청하신 메뉴를 찾을 수 없습니다."
    else:
        info_list = [extract_menu_info(doc) for doc in docs]
        response_text = "요청하신 메뉴 정보입니다:\n\n"
        for info in info_list:
            response_text += f"**- 메뉴:** {info['name']}\n"
            response_text += f"  **- 가격:** {info['price']}\n"
            response_text += f"  **- 설명:** {info['description']}\n\n"
            
    return {"messages": [AIMessage(content=response_text)]}

def handle_price_inquiry(state: MessagesState) -> dict:
    """메뉴 가격에 대한 문의를 처리합니다."""
    # '가격'과 관련된 일반적인 쿼리로 검색하여 여러 메뉴의 가격을 보여줌
    docs = cafe_db.similarity_search("메뉴 가격", k=5)
    info_list = [extract_menu_info(doc) for doc in docs]
    
    response_text = "주요 메뉴의 가격 정보입니다:\n"
    for info in info_list:
        response_text += f"- {info['name']}: {info['price']}\n"
        
    return {"messages": [AIMessage(content=response_text)]}

def handle_recommendation(state: MessagesState) -> dict:
    """메뉴 추천 요청을 처리합니다."""
    user_message = state["messages"][-1].content
    # 먼저 사용자 메시지로 검색 시도
    docs = cafe_db.similarity_search(user_message, k=3)
    
    # 결과가 없으면 '인기 메뉴'로 다시 검색
    if not docs:
        docs = cafe_db.similarity_search("인기 메뉴", k=3)
    
    info_list = [extract_menu_info(doc) for doc in docs]
    
    response_text = "이런 메뉴는 어떠세요? 😋\n\n"
    for info in info_list:
        response_text += f"**- 메뉴:** {info['name']}\n"
        response_text += f"  **- 설명:** {info['description']}\n\n"
        
    return {"messages": [AIMessage(content=response_text)]}

def handle_general_inquiry(state: MessagesState) -> dict:
    """일반적인 인사나 관련 없는 문의에 응답합니다."""
    response_text = "안녕하세요! 저희 카페에 오신 것을 환영합니다. 메뉴, 가격, 추천에 대해 무엇이든 물어보세요! 😊"
    return {"messages": [AIMessage(content=response_text)]}

In [6]:
# 라우팅 함수: 상태를 받아 다음 실행할 노드의 이름을 반환
def route_message(state: MessagesState):
    user_message = state["messages"][-1].content
    classification = classify_question(user_message)
    
    if classification == "price":
        return "price_inquiry"
    elif classification == "recommendation":
        return "recommendation"
    elif classification == "menu":
        return "menu_inquiry"
    else:
        return "general_inquiry"

# 그래프 빌더 초기화
builder = StateGraph(MessagesState)

# 노드 추가
builder.add_node("menu_inquiry", handle_menu_inquiry)
builder.add_node("price_inquiry", handle_price_inquiry)
builder.add_node("recommendation", handle_recommendation)
builder.add_node("general_inquiry", handle_general_inquiry)

# 진입점(START)에서 조건부 라우팅 설정
builder.add_conditional_edges(
    START,
    route_message,
    {
        "menu_inquiry": "menu_inquiry",
        "price_inquiry": "price_inquiry",
        "recommendation": "recommendation",
        "general_inquiry": "general_inquiry",
    }
)

# 각 노드 실행 후에는 그래프 종료(END)
builder.add_edge("menu_inquiry", END)
builder.add_edge("price_inquiry", END)
builder.add_edge("recommendation", END)
builder.add_edge("general_inquiry", END)

# 그래프 컴파일
graph = builder.compile()

print("LangGraph가 성공적으로 컴파일되었습니다.")

LangGraph가 성공적으로 컴파일되었습니다.


In [8]:
from pprint import pprint
from langchain_core.messages import HumanMessage

# --- 테스트 1: 특정 메뉴 문의 ---
print("===== [Test 1: 특정 메뉴 문의] =====")
# 입력 형식을 딕셔너리로 수정
inputs = {"messages": [HumanMessage(content="크로크무슈에 대해 알려주세요.")]}
# 그래프 호출
response = graph.invoke(inputs)
# 출력 형식도 딕셔너리이므로 'messages' 키로 접근
pprint(response["messages"][-1].content)
print("\n" + "="*30 + "\n")


# --- 테스트 2: 가격 문의 ---
print("===== [Test 2: 가격 문의] =====")
# 입력 형식을 딕셔너리로 수정
inputs = {"messages": [HumanMessage(content="커피 가격이 궁금해요")]}
response = graph.invoke(inputs)
# 출력 형식도 딕셔너리이므로 'messages' 키로 접근
pprint(response["messages"][-1].content)
print("\n" + "="*30 + "\n")


# --- 테스트 3: 추천 요청 ---
print("===== [Test 3: 추천 요청] =====")
# 입력 형식을 딕셔너리로 수정
inputs = {"messages": [HumanMessage(content="달콤한 케이크 추천해줘")]}
response = graph.invoke(inputs)
# 출력 형식도 딕셔너리이므로 'messages' 키로 접근
pprint(response["messages"][-1].content)
print("\n" + "="*30 + "\n")


# --- 테스트 4: 일반 인사 ---
print("===== [Test 4: 일반 인사] =====")
# 입력 형식을 딕셔너리로 수정
inputs = {"messages": [HumanMessage(content="안녕!")]}
response = graph.invoke(inputs)
# 출력 형식도 딕셔너리이므로 'messages' 키로 접근
pprint(response["messages"][-1].content)

===== [Test 1: 특정 메뉴 문의] =====
('요청하신 메뉴 정보입니다:\n'
 '\n'
 '**- 메뉴:** 카페라떼\n'
 '  **- 가격:** ₩5,500\n'
 '  **- 설명:** 진한 에스프레소에 부드럽게 스팀한 우유를 넣어 만든 대표적인 밀크 커피입니다. 크리미한 질감과 부드러운 맛이 '
 '특징이며, 다양한 시럽과 토핑 추가가 가능합니다. 라떼 아트로 시각적 즐거움도 제공합니다.\n'
 '\n'
 '**- 메뉴:** 프라푸치노\n'
 '  **- 가격:** ₩7,000\n'
 '  **- 설명:** 에스프레소와 우유, 얼음을 블렌더에 갈아 만든 시원한 음료입니다. 부드럽고 크리미한 질감이 특징이며, 휘핑크림을 '
 '올려 달콤함을 더했습니다. 여름철 인기 메뉴입니다.\n'
 '\n')


===== [Test 2: 가격 문의] =====
('주요 메뉴의 가격 정보입니다:\n'
 '- 녹차 라떼: ₩5,800\n'
 '- 바닐라 라떼: ₩6,000\n'
 '- 프라푸치노: ₩7,000\n'
 '- 카페라떼: ₩5,500\n'
 '- 티라미수: ₩7,500\n')


===== [Test 3: 추천 요청] =====
('이런 메뉴는 어떠세요? 😋\n'
 '\n'
 '**- 메뉴:** 티라미수\n'
 '  **- 설명:** 이탈리아 전통 디저트로 마스카포네 치즈와 에스프레소에 적신 레이디핑거를 층층이 쌓아 만들었습니다. 부드럽고 달콤한 '
 '맛이 특징이며, 코코아 파우더로 마무리하여 깊은 풍미를 자랑합니다.\n'
 '\n'
 '**- 메뉴:** 아이스 아메리카노\n'
 '  **- 설명:** 진한 에스프레소에 차가운 물과 얼음을 넣어 만든 시원한 아이스 커피입니다. 깔끔하고 시원한 맛이 특징이며, 원두 '
 '본연의 풍미를 느낄 수 있습니다. 더운 날씨에 인기가 높습니다.\n'
 '\n'
 '**- 메뉴:** 카페라떼\n'
 '  **- 설명:** 진한 에스프레소에 부드럽게 스팀한 우유를 넣어 만든 대표적인 밀크 커피입니다. 크리미한